In [22]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = '.\\t20s_csv2'

# Use glob to find all CSV files in the specified directory
info_files = glob.glob(os.path.join(directory, '*_info.csv'))
delivery_files = glob.glob(os.path.join(directory,'*.csv'))

matches=[]
deliveries=[]
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split('\\')[-1])
for delivery in delivery_files:
    if '_info' not in delivery:
        deliveries.append(delivery.split('\\')[-1])

# Matches

In [23]:
match_ids=[]
for csv_file in matches:
    match_ids.append(csv_file.split('_')[0])
    
# Define the initial and final schemas
initial_schema = {'col1': pl.String, 'attributes': pl.String, 'values': pl.String, 'players': pl.String, 'code': pl.String}
final_schema = [
    ('team1', pl.String),
    ('team2', pl.String),
    ('gender', pl.String),
    ('season', pl.String),
    ('date', pl.String),
    ('venue', pl.String),
    ('city', pl.String),
    ('toss_winner', pl.String),
    ('toss_decision', pl.String),
    ('winner', pl.String),
]

# Create a dictionary from the final schema
final_schema_dict = {key: value for key, value in final_schema}

# Initialize an empty DataFrame with the final schema
matches_data = pl.DataFrame(schema=final_schema_dict)

# List to store recalculated match IDs
recalculated_matchids = match_ids[:]

# Iterate over matches and process each one
for idx, match in enumerate(matches):
    try:
        match_df = pl.read_csv(f'./t20s_csv2/{match}', schema=initial_schema)
        
        # Extract team names
        team1_name = match_df[1, 'values']
        team2_name = match_df[2, 'values']
        
        # Replace team names
        match_df = match_df.with_columns([
            pl.when((pl.col('attributes') == 'team') & (pl.col('values') == team1_name))
            .then(pl.lit('team1'))
            .when((pl.col('attributes') == 'team') & (pl.col('values') == team2_name))
            .then(pl.lit('team2'))
            .otherwise(pl.col('attributes'))
            .alias('attributes')
        ])
        
        # Select and transpose the DataFrame
        match_transposed = match_df.select("attributes", "values").transpose(include_header=True, column_names="attributes").drop("column")
        
        # Ensure all columns in final_schema_dict are present
        missing_cols = [col for col in final_schema_dict.keys() if col not in match_transposed.columns]
        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        # Select the required columns and append to matches_data
        match_transposed = match_transposed.select(final_schema_dict.keys())
        matches_data = matches_data.vstack(match_transposed)
    except Exception as e:
        recalculated_matchids.remove(match_ids[idx])
matches_data=matches_data.with_columns(pl.Series(recalculated_matchids).alias("match_id").cast(pl.Int64))
matches_data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/17""","""Melbourne Cricket Ground""",null,"""Sri Lanka""","""field""","""Sri Lanka""",1001349
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/19""","""Simonds Stadium, South Geelong""","""Victoria""","""Sri Lanka""","""field""","""Sri Lanka""",1001351
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/22""","""Adelaide Oval""",null,"""Sri Lanka""","""field""","""Australia""",1001353
"""Ireland""","""Hong Kong""","""male""","""2016""","""2016/09/05""","""Bready Cricket Club, Magherama…","""Londonderry""","""Hong Kong""","""bat""","""Hong Kong""",1004729
"""Zimbabwe""","""India""","""male""","""2016""","""2016/06/18""","""Harare Sports Club""",null,"""India""","""field""","""Zimbabwe""",1007655
…,…,…,…,…,…,…,…,…,…,…
"""Pakistan""","""Sri Lanka""","""male""","""2015/16""","""2016/03/04""","""Shere Bangla National Stadium""","""Mirpur""","""Pakistan""","""field""","""Pakistan""",966763
"""Bangladesh""","""India""","""male""","""2015/16""","""2016/03/06""","""Shere Bangla National Stadium""","""Mirpur""","""India""","""field""","""India""",966765
"""United Arab Emirates""","""Netherlands""","""male""","""2015/16""","""2016/02/03""","""ICC Academy""","""Dubai""","""United Arab Emirates""","""field""","""Netherlands""",967081


In [24]:
players_schema=([('team1_player1', pl.String),
             ('team1_player2', pl.String),
             ('team1_player3', pl.String),
             ('team1_player4', pl.String),
             ('team1_player5', pl.String),
             ('team1_player6', pl.String),
             ('team1_player7', pl.String),
             ('team1_player8', pl.String),
             ('team1_player9', pl.String),
             ('team1_player10', pl.String),
             ('team1_player11', pl.String),
             ('team2_players1', pl.String),
             ('team2_players2', pl.String),
             ('team2_players3', pl.String),
             ('team2_players4', pl.String),
             ('team2_players5', pl.String),
             ('team2_players6', pl.String),
             ('team2_players7', pl.String),
             ('team2_players8', pl.String),
             ('team2_players9', pl.String),
             ('team2_players10', pl.String),
             ('team2_players11', pl.String)])
players_data=pl.DataFrame(schema=players_schema)
injured_matches=[]
for match_id in recalculated_matchids:
    try:
        match=pl.read_csv(f'./t20s_csv2/{match_id}_info.csv',schema=initial_schema)
        match= match.select("attributes","values","players")
        team1_players=match.filter((match['attributes']=="player"))[:11]["players"]
        team2_players=match.filter((match['attributes']=="player"))[11:]["players"]
        match_players=pl.DataFrame(team1_players).transpose(column_names=[f"team1_player{i}" for i in range(1,12)])
        match_players=match_players.hstack(pl.DataFrame(team2_players).transpose(column_names=[f"team2_players{i}" for i in range(1,12)]))
        players_data=players_data.vstack(match_players)
    except:
        injured_matches.append(match_id)
        continue
players_data

team1_player1,team1_player2,team1_player3,team1_player4,team1_player5,team1_player6,team1_player7,team1_player8,team1_player9,team1_player10,team1_player11,team2_players1,team2_players2,team2_players3,team2_players4,team2_players5,team2_players6,team2_players7,team2_players8,team2_players9,team2_players10,team2_players11
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""AJ Finch""","""M Klinger""","""TM Head""","""MC Henriques""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""A Zampa""","""B Stanlake""","""AJ Tye""","""N Dickwella""","""WU Tharanga""","""EMDY Munaweera""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""S Prasanna""","""JRMVB Sanjaya""","""SL Malinga""","""KMDN Kulasekara""","""PADLR Sandakan"""
"""M Klinger""","""AJ Finch""","""BR Dunk""","""MC Henriques""","""TM Head""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""AJ Tye""","""JA Richardson""","""N Dickwella""","""WU Tharanga""","""EMDY Munaweera""","""BKG Mendis""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""S Prasanna""","""KMDN Kulasekara""","""SL Malinga""","""JRMVB Sanjaya"""
"""AJ Finch""","""M Klinger""","""BR Dunk""","""TM Head""","""MC Henriques""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""JA Richardson""","""A Zampa""","""EMDY Munaweera""","""WU Tharanga""","""BKG Mendis""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""MD Shanaka""","""S Prasanna""","""KMDN Kulasekara""","""SL Malinga""","""JRMVB Sanjaya"""
"""Nizakat Khan""","""Aizaz Khan""","""Babar Hayat""","""Anshuman Rath""","""Ehsan Khan""","""Tanwir Afzal""","""Shahid Wasif""","""Tanveer Ahmed""","""Ehsan Nawaz""","""Nadeem Ahmed""","""C Carter""","""WTS Porterfield""","""DA Rankin""","""SP Terry""","""JNK Shannon""","""KJ O'Brien""","""GJ Thompson""","""AR McBrine""","""L Tucker""","""J Mulder""","""J Little""","""CA Young"""
"""KL Rahul""","""Mandeep Singh""","""AT Rayudu""","""MK Pandey""","""KM Jadhav""","""MS Dhoni""","""AR Patel""","""R Dhawan""","""JJ Bumrah""","""JD Unadkat""","""YS Chahal""","""CJ Chibhabha""","""H Masakadza""","""R Mutumbami""","""Sikandar Raza""","""MN Waller""","""CT Mutombodzi""","""E Chigumbura""","""AG Cremer""","""N Madziva""","""T Muzarabani""","""DT Tiripano"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sharjeel Khan""","""Mohammad Hafeez""","""Sarfraz Ahmed""","""Umar Akmal""","""Shoaib Malik""","""Iftikhar Ahmed""","""Shahid Afridi""","""Mohammad Amir""","""Wahab Riaz""","""Mohammad Irfan""","""Mohammad Nawaz (3)""","""LD Chandimal""","""TM Dilshan""","""GSNFG Jayasuriya""","""CK Kapugedera""","""MD Shanaka""","""TAM Siriwardana""","""N Dickwella""","""NLTC Perera""","""KMDN Kulasekara""","""HMRKB Herath""","""PVD Chameera"""
"""Tamim Iqbal""","""Soumya Sarkar""","""Sabbir Rahman""","""Shakib Al Hasan""","""Mushfiqur Rahim""","""Mashrafe Mortaza""","""Mahmudullah""","""Nasir Hossain""","""Al-Amin Hossain""","""Abu Hider""","""Taskin Ahmed""","""RG Sharma""","""S Dhawan""","""V Kohli""","""MS Dhoni""","""SK Raina""","""Yuvraj Singh""","""HH Pandya""","""RA Jadeja""","""R Ashwin""","""JJ Bumrah""","""A Nehra"""
"""MR Swart""","""SJ Myburgh""","""W Barresi""","""PW Borren""","""PM Seelaar""","""MP O'Dowd""","""Mudassar Bukhari""","""MJG Rippon""","""Sikander Zulfiqar""","""PA van Meekeren""","""Ahsan Malik""","""Hafiz Qaleem""","""Mohammad Shahzad (2)""","""Shaiman Anwar""","""SP Patil""","""Mohammad Usman""","""Fahad Tariq""","""Rohan Mustafa""","""Mohammad Naveed""","""Ahmed Raza""","""Farhan Ahmed""","""Qadeer Ahmed"""


In [27]:
non_injured_matches = [int(match) for match in recalculated_matchids if match not in injured_matches]
len(non_injured_matches)

3463

In [29]:
matches_data=matches_data.filter(matches_data['match_id'].is_in(non_injured_matches))
matches_data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/17""","""Melbourne Cricket Ground""",null,"""Sri Lanka""","""field""","""Sri Lanka""",1001349
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/19""","""Simonds Stadium, South Geelong""","""Victoria""","""Sri Lanka""","""field""","""Sri Lanka""",1001351
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/22""","""Adelaide Oval""",null,"""Sri Lanka""","""field""","""Australia""",1001353
"""Ireland""","""Hong Kong""","""male""","""2016""","""2016/09/05""","""Bready Cricket Club, Magherama…","""Londonderry""","""Hong Kong""","""bat""","""Hong Kong""",1004729
"""Zimbabwe""","""India""","""male""","""2016""","""2016/06/18""","""Harare Sports Club""",null,"""India""","""field""","""Zimbabwe""",1007655
…,…,…,…,…,…,…,…,…,…,…
"""Pakistan""","""Sri Lanka""","""male""","""2015/16""","""2016/03/04""","""Shere Bangla National Stadium""","""Mirpur""","""Pakistan""","""field""","""Pakistan""",966763
"""Bangladesh""","""India""","""male""","""2015/16""","""2016/03/06""","""Shere Bangla National Stadium""","""Mirpur""","""India""","""field""","""India""",966765
"""United Arab Emirates""","""Netherlands""","""male""","""2015/16""","""2016/02/03""","""ICC Academy""","""Dubai""","""United Arab Emirates""","""field""","""Netherlands""",967081


In [30]:
players_data=players_data.with_columns(pl.Series(non_injured_matches).alias("match_id").cast(pl.Int64))
players_data

team1_player1,team1_player2,team1_player3,team1_player4,team1_player5,team1_player6,team1_player7,team1_player8,team1_player9,team1_player10,team1_player11,team2_players1,team2_players2,team2_players3,team2_players4,team2_players5,team2_players6,team2_players7,team2_players8,team2_players9,team2_players10,team2_players11,match_id
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""AJ Finch""","""M Klinger""","""TM Head""","""MC Henriques""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""A Zampa""","""B Stanlake""","""AJ Tye""","""N Dickwella""","""WU Tharanga""","""EMDY Munaweera""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""S Prasanna""","""JRMVB Sanjaya""","""SL Malinga""","""KMDN Kulasekara""","""PADLR Sandakan""",1001349
"""M Klinger""","""AJ Finch""","""BR Dunk""","""MC Henriques""","""TM Head""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""AJ Tye""","""JA Richardson""","""N Dickwella""","""WU Tharanga""","""EMDY Munaweera""","""BKG Mendis""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""S Prasanna""","""KMDN Kulasekara""","""SL Malinga""","""JRMVB Sanjaya""",1001351
"""AJ Finch""","""M Klinger""","""BR Dunk""","""TM Head""","""MC Henriques""","""AJ Turner""","""JP Faulkner""","""TD Paine""","""PJ Cummins""","""JA Richardson""","""A Zampa""","""EMDY Munaweera""","""WU Tharanga""","""BKG Mendis""","""DAS Gunaratne""","""TAM Siriwardana""","""CK Kapugedera""","""MD Shanaka""","""S Prasanna""","""KMDN Kulasekara""","""SL Malinga""","""JRMVB Sanjaya""",1001353
"""Nizakat Khan""","""Aizaz Khan""","""Babar Hayat""","""Anshuman Rath""","""Ehsan Khan""","""Tanwir Afzal""","""Shahid Wasif""","""Tanveer Ahmed""","""Ehsan Nawaz""","""Nadeem Ahmed""","""C Carter""","""WTS Porterfield""","""DA Rankin""","""SP Terry""","""JNK Shannon""","""KJ O'Brien""","""GJ Thompson""","""AR McBrine""","""L Tucker""","""J Mulder""","""J Little""","""CA Young""",1004729
"""KL Rahul""","""Mandeep Singh""","""AT Rayudu""","""MK Pandey""","""KM Jadhav""","""MS Dhoni""","""AR Patel""","""R Dhawan""","""JJ Bumrah""","""JD Unadkat""","""YS Chahal""","""CJ Chibhabha""","""H Masakadza""","""R Mutumbami""","""Sikandar Raza""","""MN Waller""","""CT Mutombodzi""","""E Chigumbura""","""AG Cremer""","""N Madziva""","""T Muzarabani""","""DT Tiripano""",1007655
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sharjeel Khan""","""Mohammad Hafeez""","""Sarfraz Ahmed""","""Umar Akmal""","""Shoaib Malik""","""Iftikhar Ahmed""","""Shahid Afridi""","""Mohammad Amir""","""Wahab Riaz""","""Mohammad Irfan""","""Mohammad Nawaz (3)""","""LD Chandimal""","""TM Dilshan""","""GSNFG Jayasuriya""","""CK Kapugedera""","""MD Shanaka""","""TAM Siriwardana""","""N Dickwella""","""NLTC Perera""","""KMDN Kulasekara""","""HMRKB Herath""","""PVD Chameera""",966763
"""Tamim Iqbal""","""Soumya Sarkar""","""Sabbir Rahman""","""Shakib Al Hasan""","""Mushfiqur Rahim""","""Mashrafe Mortaza""","""Mahmudullah""","""Nasir Hossain""","""Al-Amin Hossain""","""Abu Hider""","""Taskin Ahmed""","""RG Sharma""","""S Dhawan""","""V Kohli""","""MS Dhoni""","""SK Raina""","""Yuvraj Singh""","""HH Pandya""","""RA Jadeja""","""R Ashwin""","""JJ Bumrah""","""A Nehra""",966765
"""MR Swart""","""SJ Myburgh""","""W Barresi""","""PW Borren""","""PM Seelaar""","""MP O'Dowd""","""Mudassar Bukhari""","""MJG Rippon""","""Sikander Zulfiqar""","""PA van Meekeren""","""Ahsan Malik""","""Hafiz Qaleem""","""Mohammad Shahzad (2)""","""Shaiman Anwar""","""SP Patil""","""Mohammad Usman""","""Fahad Tariq""","""Rohan Mustafa""","""Mohammad Naveed""","""Ahmed Raza""","""Farhan Ahmed""","""Qadeer Ahmed""",967081


In [32]:
delivery_schema=([('match_id', pl.Int64),
                ('season', pl.String),
                ('start_date', pl.String),
                ('venue', pl.String),
                ('innings', pl.Int64),
                ('ball', pl.Float64),
                ('batting_team', pl.String),
                ('bowling_team', pl.String),
                ('striker', pl.String),
                ('non_striker', pl.String),
                ('bowler', pl.String),
                ('runs_off_bat', pl.Int64),
                ('extras', pl.Int64),
                ('wides', pl.Int64),
                ('noballs', pl.String),
                ('byes', pl.Int64),
                ('legbyes', pl.Int64),
                ('penalty', pl.String),
                ('wicket_type', pl.String),
                ('player_dismissed', pl.String),
                ('other_wicket_type', pl.String),
                ('other_player_dismissed', pl.String)])
delivery_schema_dict = {key : value for key,value in delivery_schema}
deliveries_data = pl.DataFrame(schema=delivery_schema_dict)
for delivery in deliveries:
    delivery=pl.read_csv('./t20s_csv2/'+delivery,schema=delivery_schema_dict)
    if delivery["match_id"][0] in non_injured_matches:
        deliveries_data=deliveries_data.vstack(delivery)
    else:
        continue
deliveries_data

match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
i64,str,str,str,i64,f64,str,str,str,str,str,i64,i64,i64,str,i64,i64,str,str,str,str,str
1001349,"""2016/17""","""2017-02-17""","""Melbourne Cricket Ground""",1,0.1,"""Australia""","""Sri Lanka""","""AJ Finch""","""M Klinger""","""SL Malinga""",0,0,null,null,null,null,null,null,null,null,null
1001349,"""2016/17""","""2017-02-17""","""Melbourne Cricket Ground""",1,0.2,"""Australia""","""Sri Lanka""","""AJ Finch""","""M Klinger""","""SL Malinga""",0,0,null,null,null,null,null,null,null,null,null
1001349,"""2016/17""","""2017-02-17""","""Melbourne Cricket Ground""",1,0.3,"""Australia""","""Sri Lanka""","""AJ Finch""","""M Klinger""","""SL Malinga""",1,0,null,null,null,null,null,null,null,null,null
1001349,"""2016/17""","""2017-02-17""","""Melbourne Cricket Ground""",1,0.4,"""Australia""","""Sri Lanka""","""M Klinger""","""AJ Finch""","""SL Malinga""",2,0,null,null,null,null,null,null,null,null,null
1001349,"""2016/17""","""2017-02-17""","""Melbourne Cricket Ground""",1,0.5,"""Australia""","""Sri Lanka""","""M Klinger""","""AJ Finch""","""SL Malinga""",0,0,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
995469,"""2016""","""2016-09-09""","""R Premadasa Stadium""",2,17.1,"""Australia""","""Sri Lanka""","""TM Head""","""PM Nevill""","""SS Pathirana""",1,0,null,null,null,null,null,null,null,null,null
995469,"""2016""","""2016-09-09""","""R Premadasa Stadium""",2,17.2,"""Australia""","""Sri Lanka""","""PM Nevill""","""TM Head""","""SS Pathirana""",3,0,null,null,null,null,null,null,null,null,null
995469,"""2016""","""2016-09-09""","""R Premadasa Stadium""",2,17.3,"""Australia""","""Sri Lanka""","""TM Head""","""PM Nevill""","""SS Pathirana""",0,0,null,null,null,null,null,null,null,null,null


In [36]:
deliveries_data.write_parquet("./historical_data/cricsheetSource/deliveries.parquet")
matches_data.write_parquet('./historical_data/cricsheetSource/matches.parquet')
players_data.write_parquet('./historical_data/cricsheetSource/match_players.parquet')

In [33]:
deliveries_data.shape,matches_data.shape,players_data.shape

((794009, 22), (3463, 11), (3463, 23))

In [34]:
matches_data[['team1','team2','gender','date','toss_winner','toss_decision','winner']].unique()

team1,team2,gender,date,toss_winner,toss_decision,winner
str,str,str,str,str,str,str
"""West Indies""","""New Zealand""","""male""","""2014/07/06""","""New Zealand""","""field""","""West Indies"""
"""Australia""","""South Africa""","""male""","""2009/01/13""","""South Africa""","""bat""","""Australia"""
"""Panama""","""Cayman Islands""","""male""","""2023/09/30""","""Panama""","""bat""","""Cayman Islands"""
"""United States of America""","""Brazil""","""female""","""2023/09/05""","""Brazil""","""field""","""United States of America"""
"""Czech Republic""","""Austria""","""male""","""2022/07/09""","""Austria""","""field""","""Austria"""
…,…,…,…,…,…,…
"""Tanzania""","""Hong Kong""","""female""","""2023/11/18""","""Tanzania""","""bat""","""Hong Kong"""
"""Italy""","""Jersey""","""male""","""2023/07/23""","""Jersey""","""field""","""Italy"""
"""England""","""Pakistan""","""male""","""2012/02/23""","""England""","""field""","""Pakistan"""
